# Libraries

In [ ]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup

# Obtendo Conexão

In [ ]:
req = requests.get("https://sol.sbc.org.br/busca/index.php/integrada/results?query=&archiveIds%5B%5D=1&archiveIds%5B%5D=2&archiveIds%5B%5D=3&isAdvanced=1&field-10%5B%5D=por&field-10%5B%5D=eng&searchPage=1#records")
if  req.status_code == 200:
    print("Conexão bem sucedida!")

In [ ]:
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())

Obtendo a última página

In [ ]:
pagination_div = soup.find("div", class_="pagination_results")

for a in pagination_div.find_all("a", href=True):
    link = a["href"]

print(link)    

In [ ]:
start_delimiter = "searchPage="
end_delimiter = "#records"


start_index = link.find(start_delimiter)
if start_index != -1:
    start_index += len(start_delimiter)
    
    end_index = link.find(end_delimiter, start_index)
    
    if end_index != -1:
        last_page = link[start_index:end_index]
        print(f"Última Página: {last_page}")
    else:
        print("Delimitador final não encontrado.")
else:
    print("Delimitador inicial não encontrado.")



Obtendo Informações dos artigos

In [ ]:
page = 1

df = pd.DataFrame(columns=['Category', 'Title', 'URL_Title', 'Authors', 'Event', 'Date'])

for page in range(int(last_page)):
    
    url = "https://sol.sbc.org.br/busca/index.php/integrada/results?query=&archiveIds%5B%5D=1&archiveIds%5B%5D=2&archiveIds%5B%5D=3&isAdvanced=1&field-10%5B%5D=por&field-10%5B%5D=eng&searchPage={page}"
    print("Página", page, " - ", url)
    
    req = requests.get(url)
    if  req.status_code == 200:
        print("Conexão bem sucedida!")
    
    soup = BeautifulSoup(req.content, 'html.parser')
    #print(soup.prettify())

    list_category = soup.find_all(class_="archive_title")
    list_title = soup.find_all(class_="title")
    list_url_title = soup.find_all(class_="record_title")
    list_authors = soup.find_all(class_="author")
    list_event = soup.find_all(class_="archive_serie")
    list_date = soup.find_all(class_="list_record_date")
 
    for category, title, url_title, authors, event, date in zip(list_category, list_title, list_url_title, list_authors, list_event, list_date):
        category = category.get_text()
        title = title.get_text()
        url_title = url_title.get('href')
        authors = authors.get_text()
        event = event.get_text()    
        date = date.get_text(strip=True)
    
        df = df._append({'Category': category, 'Title': title, 'URL_Title': url_title, 'Authors': authors, 'Event': event, 'Date': date}, ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('df_artigos.csv', index=True)

Obtendo os PDF de cada artigo

In [ ]:
df_papers = pd.DataFrame(columns=['Event', 'Authors', 'Title', 'Abstract', 'Keywords', 'Publisher', 'URL_Paper', 'Date'])

for url in df['URL_Title']:
    print(url)

    req = requests.get(url)

    soup = BeautifulSoup(req.content, 'html.parser')
    soup.prettify()

    event = soup.find(class_="cmp_breadcrumbs")
    event = event.get_text()
    #print(event)

    list_authors = soup.find(class_="item authors")
    list_authors = list_authors.get_text(strip=True)
    #print(list_authors)

    title = soup.find(class_="page_title")
    title = title.get_text(strip=True)
    #print(title)

    abstract = soup.find(class_="item abstract")
    abstract = abstract.get_text(strip=True)
    #print(abstract)

    try:
        keywords = soup.find(class_="item keywords")
        keywords = keywords.get_text(strip=True)
    except AttributeError:
        keywords = "No keywords available"
    #print(keywords)

    try:
        url_paper = (soup.find(class_="obj_galley_link pdf") or soup.find(class_="obj_galley_link file"))
        publisher = url_paper.get_text(strip=True)
        #print(publisher)

        url_paper = url_paper.get('href')
        #print(url_paper)
    
    except AttributeError:
        publisher = "No publisher available"
        url_paper = "No URL available"

    date = soup.find(class_="item published")
    date = date.get_text(strip=True)
    #print(date)

    df_papers = df_papers._append({'Event': event, 
                     'Authors': list_authors, 
                     'Title': title, 
                     'Abstract' : abstract, 
                     'Keywords' : keywords, 
                     'Publisher' : publisher, 
                     'URL_Paper' : url_paper,
                     'Date': date}, 
                     
                     ignore_index=True)

df_papers.to_csv('df_papers.csv', index=True)

    

In [ ]:
df_papers.info()

In [ ]:
pdf_link = "https://sol.sbc.org.br/index.php/sbrlars/article/download/39269/39041"

pdf_bytes = requests.get(pdf_link).content
with open("artigo_29986.pdf", "wb") as f:
    f.write(pdf_bytes)